In [ ]:
# This page is for provisioning an ESP32 GPS device for the wingtip or base-station.  
# This interfaces with the Hanglog3 android app to record the data, which the 
# scripts in this repository can interact with to enable RTK technology


In [26]:
#### esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/executables/micropythonbins/esp32-idf4-20200124-v1.12-87-g96716b46e.bin
%esptool esp32 /home/julian/executables/micropythonbins/esp32-idf4-20200701-unstable-v1.12-606-g07f181a21.bin



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7f3f0c053e90, open=True>(port='/dev/ttyUSB1', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Executing:
  esptool.py --port /dev/ttyUSB0 --chip esp32 write_flash -z 0x1000 /home/julian/executables/micropythonbins/esp32-idf4-20200701-unstable-v1.12-606-g07f181a21.bin

esptool.py v2.8
Serial port /dev/ttyUSB0
Connecting....
[Press the PRG button now if required]
Chip is ESP32D0WDQ6 (revision 1)
Features: WiFi, BT, Dual Core, 240MHz, VRef calibration in efuse, Coding Scheme None
Crystal is 40MHz
MAC: 3c:71:bf:ff:67:7c
Uploading stub...
Running stub...
Stub running...
Configuring flash size...
Auto-detected Flash size: 4MB
Compressed 1448848 bytes to 924753...
Wrote 1448848 bytes (924753 compressed) at 0x00001000 in 81.9 seconds (effective 141.5 kbit/s)...
Hash of data verified.

Leaving...
Hard resetting via RTS pin...


In [27]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [28]:
help("modules")

__main__          gc                uasyncio/stream   upip_utarfile
_boot             inisetup          ubinascii         upysh
_onewire          machine           ubluetooth        urandom
_thread           math              ucollections      ure
_uasyncio         micropython       ucryptolib        urequests
_webrepl          neopixel          uctypes           uselect
apa106            network           uerrno            usocket
btree             ntptime           uhashlib          ussl
builtins          onewire           uhashlib          ustruct
cmath             sys               uheapq            utime
dht               uarray            uio               utimeq
ds18x20           uasyncio/__init__ ujson             uwebsocket
esp               uasyncio/core     umqtt/robust      uzlib
esp32             uasyncio/event    umqtt/simple      webrepl
flashbdev         uasyncio/funcs    uos               webrepl_setup
framebuf          uasyncio/lock     upip              websocket_hel

In [29]:
%sendtofile --source utils.py
%sendtofile --source ubxm8t.py

Sent 66 lines (2004 bytes) to ubxm8t.py.


In [34]:
%sendtofile config.txt

deviceletter  C
pinled        2
pingled       23
connection1   BV6000,beckaaaa,192.168.43.1,9042
connection0   JulianS9,beckaaaa,192.168.43.1,9042
sdcard        sd,5,18,23,19

Sent 6 lines (174 bytes) to config.txt.


In [48]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [50]:
%sendtofile --source main.py

Sent 105 lines (3616 bytes) to main.py.


In [47]:
%sendtofile --source utils.py

Sent 94 lines (2645 bytes) to utils.py.


In [ ]:
#
# Code below is for developing and testing the components
#

In [4]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [44]:
import uasyncio as asyncio
import network

si = network.WLAN(network.STA_IF)
si.active(True)

async def wifi_connect(self):
    s.active(True)
    s.connect(self._ssid, self._wifi_pw)
    while s.status() == network.STAT_CONNECTING:  # Break out on fail or success. Check once per sec.
        await asyncio.sleep(1)

    if not s.isconnected():
        raise OSError
    # Ensure connection stays up for a few secs.
    self.dprint('Checking WiFi integrity.')
    for _ in range(5):
        if not s.isconnected():
            raise OSError  # in 1st 5 secs
        await asyncio.sleep(1)
    self.dprint('Got reliable connection')

Sent 281 lines (8634 bytes) to sdcard.py.


In [12]:
# for testing UBX is wired in properly
import machine, time
uartUBX = machine.UART(1, baudrate=9600, rx=16, tx=17)
for i in range(20):
    time.sleep(0.2)
    print(uartUBX.readline())

In [5]:
# for testing the adc pin for measuring battery power
from machine import ADC, Pin
import time

a = ADC(Pin(34))
a.atten(ADC.ATTN_11DB)
a.width(ADC.WIDTH_12BIT)
for i in range(20):
    print(a.read())
    time.sleep(1)

983
976
977
976
976
977
2318
2313
2307
2304
.2306
2260
976
976
976
977
976
978
977
981
.

In [23]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [17]:
%ls

Listing directory '/'.
     1067    UDPblackviewphone.py
      870    UDPsocket_funcs.py
      139    boot.py
        1    deviceletter.txt
      205    hotspots.txt
     2913    main.py
     2004    ubxminimal.py
     5667    ubxutils.py


In [18]:
%fetchfile --print deviceletter.txt

B

In [24]:
# For testing the SD card
%sendtofile --source sdcard.py

Sent 281 lines (8634 bytes) to sdcard.py.


In [25]:
# External SPI work:

# card facing up:  Pin top right: VIN
# Pins on left:
# G       
# SS     ESP-GPIO5
# SCLK   ESP-GPIO18
# MOSI   ESP-GPIO23
# MISO   ESP-GPIO12


import machine, sdcard, os
from machine import Pin, SPI
spisd = SPI(-1, sck=Pin(18), mosi=Pin(23), miso=Pin(19))
sd = sdcard.SDCard(spisd, machine.Pin(5))
os.mount(sd, '/sd')
print(os.listdir('/sd'))


Traceback (most recent call last):
  File "<stdin>", line 15, in <module>
  File "sdcard.py", line 54, in __init__
  File "sdcard.py", line 82, in init_card
OSError: no SD card


In [13]:
help(SPI)

object <class 'SoftSPI'> is of type type
  init -- <function>
  deinit -- <function>
  read -- <function>
  readinto -- <function>
  write -- <function>
  write_readinto -- <function>
  MSB -- 0
  LSB -- 1


In [9]:
print(os.listdir('/sd/OOH'))


['000.TXT', '001.TXT']


In [45]:
ss = cc

In [49]:
import sdcard, os
sd = sdcard.SDCard(hspi, ss)
os.mount(sd, '/sd2')
print(os.listdir('/sd2'))

In [10]:
x = open('/sd/OOH/001.TXT')
for l in x:
    print(l, end="")

Logfile: OOH/001.TXT  Date: 2015-08-12T13:16:18
Device number: 1
Compile date: Aug 12 2015  13:16:15
Absolute GPS position:
  Qt[8HEX milliseconds timstamp]   u[8HEX milliseconds after midnight]   y[8HEX lat*600000]   x[8HEX lng*600000]   v[4HEX velocity knots*100]   d[4HEX velocity direction degrees*100]
Orient reading:
  Ot[8HEX milliseconds]   w[4HEX x-vector]   x[4HEX x-vector]   y[4HEX y-vector]   z[4HEX z-vector]

Dt00001321i00r0127
Ht00001339r8052a0000
Bt00001388s25p0001893Dc07C9
Bt000013E2s29p0001893Cc07CB
Bt00001426s2Cp0001893Ac07CC
Bt00001480s30p0001893Ec07CE
Bt000014ADs32p0001893Bc07CE
Bt000014C4s33p0001893Bc07CF
Bt000014DAs34p0001893Dc07CF
Bt000014F1s35p0001893Cc07CF
Ht00001504r8052a5F4C
Bt0000151Es37p0001893Bc07D0
Dt0000162Bi00r0127
Bt000016B4s09p0001893Bc07D2
Ht000016CEr8036a5F50
Bt00001768s11p0001893Cc07D3
Bt000017D9s16p0001893Fc07D4
Bt00001860s1Cp0001893Fc07D5
Ht00001899r801Aa5F54
Dt00001933i00r0128
Bt0000199Cs2Ap0001893Cc07D6
Ht00001A63r801Aa5F50
Bt00001A67s33p0001893F

In [12]:
%ls /sd/OOH

Listing directory '/sd/OOH'.
      424    sd/OOH/000.TXT
     3846    sd/OOH/001.TXT


In [14]:
%ls

Listing directory '/'.
     3251    Dmain.py
      139    boot.py
        1    deviceletter.txt
       93    hotspots.txt
             sd/
     8634    sdcard.py
     2004    ubxm8t.py
     1787    utils.py


In [15]:
print(open("deviceletter.txt", "r").read())

C


In [74]:
print(a.read())

1872


In [63]:
help(ADC)

object <class 'ADC'> is of type type
  read_u16 -- <function>
  read -- <function>
  atten -- <function>
  width -- <classmethod>
  ATTN_0DB -- 0
  ATTN_2_5DB -- 1
  ATTN_6DB -- 2
  ATTN_11DB -- 3
  WIDTH_9BIT -- 0
  WIDTH_10BIT -- 1
  WIDTH_11BIT -- 2
  WIDTH_12BIT -- 3


In [64]:
print(a.atten(ADC.ATTN_11DB))

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
TypeError: function takes 2 positional arguments but 1 were given
